In [0]:
%run "/Users/freestyleabg@gmail.com/batch_processing_pipeline"

/mnt/user-0a5040edb649-bucket already mounted. You can unmount using dbutils.fs.unmount('/mnt/user-0a5040edb649-bucket') if needed.

Out[6]: [FileInfo(path='dbfs:/mnt/user-0a5040edb649-bucket/topics/0a5040edb649.geo/', name='0a5040edb649.geo/', size=0, modificationTime=1714532714616),
 FileInfo(path='dbfs:/mnt/user-0a5040edb649-bucket/topics/0a5040edb649.pin/', name='0a5040edb649.pin/', size=0, modificationTime=1714532714616),
 FileInfo(path='dbfs:/mnt/user-0a5040edb649-bucket/topics/0a5040edb649.user/', name='0a5040edb649.user/', size=0, modificationTime=1714532714616)]

key,value
spark.databricks.delta.formatCheck.enabled,false


### Find the most popular category people post to based on their country.

In [0]:
from pyspark.sql.window import Window
# Join df_pin and df_geo dataframes on index coumn
df_pin_geo = df_pin.join(df_geo, df_pin.ind == df_geo.ind)

# Define a window function to partition by country and order by category_count in descending order.
windowCategoryPerCountry = Window.partitionBy("country").orderBy(col("category_count").desc())

# Group the data by "country" and "category", and count the number of posts in each category.
# Add a column "category_count" to store the count.
# Add a rank column using row_number() function over the defined window function.
# Filter the rows where rank is 1 to get the category with the highest post count for each country.
# Drop the rank column and display the result.
df_pin_geo.groupBy("country", "category") \
    .agg(count("category").alias("category_count")) \
    .withColumn("rank", row_number().over(windowCategoryPerCountry)) \
    .filter(col("rank") == 1) \
    .drop("rank") \
    .show()

+--------------------+--------------+--------------+
 country| category|category_count|
+--------------------+--------------+--------------+
 Albania| mens-fashion| 180|
 Angola|diy-and-crafts| 6|
 Anguilla| home-decor| 8|
Antarctica (the t...| christmas| 2|
 Antigua and Barbuda| quotes| 2|
 Armenia|diy-and-crafts| 170|
 Aruba| tattoos| 35|
 Australia| mens-fashion| 2|
 Austria| travel| 2|
 Azerbaijan|event-planning| 8|
 Bangladesh| art| 2|
 Barbados| travel| 6|
 Botswana| tattoos| 2|
 Bulgaria| finance| 15|
 Cambodia|diy-and-crafts| 8|
Central African R...| christmas| 2|
Cocos (Keeling) I...| vehicles| 15|
 Colombia| finance| 99|
 Cote d'Ivoire| education| 15|
 Dominican Republic| travel| 2|
+--------------------+--------------+--------------+
only showing top 20 rows

### Find how many posts each category had between 2018 and 2022

In [0]:
# Define a window function to partition by post_year and order by category_count in descending order.
windowCategoryPerYear = Window.partitionBy("post_year").orderBy(col("category_count").desc())

# Add a new column "post_year" to df_pin_geo by extracting the year from the "timestamp" column.
# Group the data by "post_year" and "category", and count the number of posts in each category.
# Add a rank column using row_number() function over the defined window function.
# Filter the rows where rank is 1 to get the category with the highest post count in each year.
# Drop the rank column and display the result.
df_pin_geo.withColumn("post_year", year("timestamp")) \
    .groupBy("post_year", "category") \
    .agg(count("category").alias("category_count")) \
    .withColumn("rank", row_number().over(windowCategoryPerYear)) \
    .filter(col("rank") == 1) \
    .drop("rank") \
    .show()

+---------+------------+--------------+
post_year| category|category_count|
+---------+------------+--------------+
 2018| beauty| 24|
 2019| quotes| 48|
 2020|mens-fashion| 180|
 2021| finance| 114|
 2022| vehicles| 15|
+---------+------------+--------------+

### For each country, find the user with the most followers.
### Then, find the country with the user with the most followers.

In [0]:

# Define a window function to partition by country and order by follower_count in descending order.
windowFollowersPerCountry = Window.partitionBy("country").orderBy(col("follower_count").desc())

# Select the columns "country", "poster_name", "follower_count" from df_pin_geo.
# Add a rank column using row_number() function over the defined window function.
# Filter the rows where rank is 1 to get the user with the most followers for each country.
# Drop the rank column and rows with null values.
most_followers_per_country = df_pin_geo \
    .select("country", "poster_name", "follower_count") \
    .withColumn("rank", row_number().over(windowFollowersPerCountry)) \
    .filter(col("rank") == 1) \
    .drop("rank") \
    .dropna()

# Order the most_followers_per_country DataFrame by follower_count in descending order
# and select the row with the highest follower_count using limit(1).
most_popular_user = most_followers_per_country \
    .orderBy(col("follower_count").desc()) \
    .limit(1)

# Display the most_popular_user DataFrame.
most_popular_user.show()

+----------+---------------+--------------+
 country| poster_name|follower_count|
+----------+---------------+--------------+
Azerbaijan|Style Me Pretty| 6000000|
+----------+---------------+--------------+

### Find the most popular category people post to per age group

In [0]:
# Define a window function to partition by age_group and order by category_count in descending order.
windowCategoryPerAge = Window.partitionBy("age_group").orderBy(col("category_count").desc())

# Join df_pin, df_geo, and df_user on the "ind" column to get the required data in df_pin_geo_user.
df_pin_geo_user = df_pin.join(df_geo, on="ind").join(df_user, on="ind")

# Add a new column "age_group" to df_pin_geo_user based on the age range of each user.
# Group the data by "age_group" and "category", and count the number of posts in each category.
# Add a rank column using row_number() function over the defined window function.
# Filter the rows where rank is 1 to get the category with the highest post count in each age group.
# Drop the rank column and display the result.
df_pin_geo_user.withColumn("age_group", when((col("age") >= 18) & (col("age") <= 24), "18-24")
                                   .when((col("age") >= 25) & (col("age") <= 35), "25-35")
                                   .when((col("age") >= 36) & (col("age") <= 50), "36-50")
                                   .when((col("age") > 50), "50+")) \
    .groupBy("age_group", "category") \
    .agg(count("category").alias("category_count")) \
    .withColumn("rank", row_number().over(windowCategoryPerAge)) \
    .filter(col("rank") == 1) \
    .drop("rank") \
    .show()

+---------+--------------+--------------+
age_group| category|category_count|
+---------+--------------+--------------+
 18-24| mens-fashion| 2178|
 25-35|diy-and-crafts| 2032|
 36-50| finance| 891|
 50+| beauty| 2|
+---------+--------------+--------------+

### Find the most popular category for different age groups

In [0]:
# Join df_pin and df_user DataFrames on the "ind" column to get the required data in df_pin_user.
df_pin_user = df_pin.join(df_user, on="ind")

# Add a new column "age_group" to df_pin_user based on the age range of each user.
df_pin_user_age = df_pin_user.withColumn("age_group", when((col("age") >= 18) & (col("age") <= 24), "18-24")
                                   .when((col("age") >= 25) & (col("age") <= 35), "25-35")
                                   .when((col("age") >= 36) & (col("age") <= 50), "36-50")
                                   .when((col("age") > 50), "50+"))

# Select the columns "age_group" and "follower_count" from df_pin_user_age DataFrame.
# Group the data by "age_group" and calculate the approximate median follower count using percentile_approx() function.
# Rename the aggregated column to "median_follower_count".
# Order the result DataFrame by "median_follower_count" in descending order and display the result.
df_pin_user_age.select("age_group", "follower_count") \
    .distinct() \
    .groupBy("age_group") \
    .agg(percentile_approx(col("follower_count"), 0.5).alias("median_follower_count")) \
    .orderBy(col("median_follower_count").desc()) \
    .show()

+---------+---------------------+
age_group|median_follower_count|
+---------+---------------------+
 18-24| 211000|
 25-35| 43000|
 36-50| 6000|
 50+| 0|
+---------+---------------------+

### Find the number of users who joined between 2015 and 2020.

In [0]:
# Join df_pin and df_user DataFrames on "ind" column to get the required data in df_pin_user.
df_pin_user = df_pin.join(df_user, on="ind")

# Add a new column "join_year" to df_pin_user that contains only the year from the "date_joined" column.
df_pin_user_year = df_pin_user.withColumn("join_year", year("date_joined"))

# Select distinct columns "user_name" and "join_year" from df_pin_user_year DataFrame.
# Group the data by "join_year" and calculate the count of join_years using count() function.
# Rename the aggregated column to "number_users_joined".
# Display the result DataFrame.
df_pin_user_year.select("user_name", "join_year") \
    .distinct() \
    .groupBy("join_year") \
    .agg(count(col("join_year")).alias("number_users_joined")) \
    .show()
    
# Print the total number of distinct user_names from df_pin_user_year DataFrame.
print(df_pin_user_year.select("user_name").distinct().count())

+---------+-------------------+
join_year|number_users_joined|
+---------+-------------------+
 2015| 16|
 2016| 12|
 2017| 3|
+---------+-------------------+

31

### Find the median follower count of users based on their joining year

In [0]:
# Join df_pin and df_user DataFrames on "ind" column to get the required data in df_pin_user.
df_pin_user = df_pin.join(df_user, on="ind")

# Add a new column "join_year" to df_pin_user that contains only the year from the "date_joined" column.
df_pin_user_year = df_pin_user.withColumn("join_year", year("date_joined"))

# Add a new column "age_group" to df_pin_user_year based on the age range of each user.
df_pin_user_year_age = df_pin_user_year.withColumn("age_group", when((col("age") >= 18) & (col("age") <= 24), "18-24")
                                   .when((col("age") >= 25) & (col("age") <= 35), "25-35")
                                   .when((col("age") >= 36) & (col("age") <= 50), "36-50")
                                   .when((col("age") > 50), "50+"))

# Select the columns "user_name, age_group", "join_year" and "follower_count" from df_pin_user_year_age DataFrame.
# Filter the data for join_years between 2015 and 2020.
# Group the data by "join_year" and calculate the approximate median follower count using percentile_approx() function.
# Rename the aggregated column to "median_follower_count".
# Order the result DataFrame by "join_year" and display the result.
df_pin_user_year_age.select("user_name", "age_group", "join_year", "follower_count") \
    .distinct() \
    .filter((col("join_year") >= 2015) & (col("join_year") <= 2020)) \
    .groupBy("join_year") \
    .agg(percentile_approx(col("follower_count"), 0.5).alias("median_follower_count")) \
    .orderBy("join_year") \
    .show()

+---------+---------------------+
join_year|median_follower_count|
+---------+---------------------+
 2015| 92000|
 2016| 21000|
 2017| 6000|
+---------+---------------------+

### Find the median follower count of users based on their joining year and age group

In [0]:
# Join df_pin and df_user DataFrames on "ind" column to get the required data in df_pin_user.
df_pin_user = df_pin.join(df_user, on="ind")

# Add a new column "join_year" to df_pin_user that contains only the year from the "date_joined" column.
df_pin_user_year = df_pin_user.withColumn("join_year", year("date_joined"))

# Add a new column "age_group" to df_pin_user_year based on the age range of each user.
df_pin_user_year_age = df_pin_user_year.withColumn("age_group", when((col("age") >= 18) & (col("age") <= 24), "18-24")
                                   .when((col("age") >= 25) & (col("age") <= 35), "25-35")
                                   .when((col("age") >= 36) & (col("age") <= 50), "36-50")
                                   .when((col("age") > 50), "50+"))

# Select the columns "user_name", "age_group", "join_year" and "follower_count" from df_pin_user_year_age DataFrame.
# Filter the data for join_years between 2015 and 2020.
# Group the data by "age_group" and "join_year" and calculate the approximate median follower count using percentile_approx() function.
# Rename the aggregated column to "median_follower_count".
# Order the result DataFrame by "age_group" and "join_year" and display the result.
df_pin_user_year_age.select("user_name", "age_group", "join_year", "follower_count") \
    .distinct() \
    .filter((col("join_year") >= 2015) & (col("join_year") <= 2020)) \
    .groupBy("age_group", "join_year") \
    .agg(percentile_approx(col("follower_count"), 0.5).alias("median_follower_count")) \
    .orderBy("age_group", "join_year") \
    .show()

+---------+---------+---------------------+
age_group|join_year|median_follower_count|
+---------+---------+---------------------+
 18-24| 2015| 211000|
 18-24| 2016| 21000|
 18-24| 2017| 940|
 25-35| 2015| 51000|
 25-35| 2016| 43000|
 25-35| 2017| 8000|
 36-50| 2015| 0|
 36-50| 2017| 6000|
 50+| 2016| 0|
+---------+---------+---------------------+